### 영어 숫자 특수문자 제거

In [10]:
import pandas as pd
import re

# 데이터 불러오기
df_dovish = pd.read_csv("dovish_filtered_no_english.csv")
df_hawkish = pd.read_csv("hawkish_filtered_no_english.csv")

# ❗ 컬럼명 직접 지정
dovish_column = "Dovish"
hawkish_column = "Hawkish"

# 영어, 숫자, 특수문자가 포함된 행을 제거하는 함수
def remove_unwanted_rows(df, column_name):
    initial_count = len(df)  # 원래 행 개수
    df_filtered = df[~df[column_name].astype(str).str.contains(r'[a-zA-Z0-9\W]', regex=True)]
    removed_count = initial_count - len(df_filtered)  # 삭제된 행 개수
    return df_filtered, removed_count

# 필터링 적용
df_dovish, removed_dovish = remove_unwanted_rows(df_dovish, dovish_column)
df_hawkish, removed_hawkish = remove_unwanted_rows(df_hawkish, hawkish_column)

# 필터링된 데이터 저장
df_dovish.to_csv("dovish_filtered_clean.csv", index=False, encoding="utf-8-sig")
df_hawkish.to_csv("hawkish_filtered_clean.csv", index=False, encoding="utf-8-sig")

# 결과 출력
print(f"비둘기파 데이터에서 삭제된 단어 개수: {removed_dovish}")
print(f"매파 데이터에서 삭제된 단어 개수: {removed_hawkish}")
print("영어, 숫자, 특수문자 포함된 행 제거 및 저장 완료!")


비둘기파 데이터에서 삭제된 단어 개수: 15
매파 데이터에서 삭제된 단어 개수: 24
영어, 숫자, 특수문자 포함된 행 제거 및 저장 완료!


## 해당 키워드 포함단어 삭제

In [20]:
import pandas as pd

# 데이터 불러오기
df_dovish = pd.read_csv("final_dovish.csv")
df_hawkish = pd.read_csv("final_hawkish.csv")

# ❗ 컬럼명 직접 지정
dovish_column = "Dovish"
hawkish_column = "Hawkish"

# "한국"이 포함된 단어를 모두 제거하는 함수
def remove_korea_related_words(df, column_name):
    initial_count = len(df)  # 원래 행 개수
    pattern = r"홀딩스"  # '한국'이 포함된 단어를 찾기 위한 패턴
    df_filtered = df[~df[column_name].astype(str).str.contains(pattern, regex=True)]
    removed_count = initial_count - len(df_filtered)  # 삭제된 행 개수
    return df_filtered, removed_count

# 필터링 적용
df_dovish, removed_dovish = remove_korea_related_words(df_dovish, dovish_column)
df_hawkish, removed_hawkish = remove_korea_related_words(df_hawkish, hawkish_column)

# 필터링된 데이터 저장
df_dovish.to_csv("final_dovish.csv", index=False, encoding="utf-8-sig")
df_hawkish.to_csv("final_hawkish.csv", index=False, encoding="utf-8-sig")

# 결과 출력
print(f"비둘기파 데이터에서 '한국' 포함 단어 삭제 개수: {removed_dovish}")
print(f"매파 데이터에서 '한국' 포함 단어 삭제 개수: {removed_hawkish}")
print("한국 포함 단어 제거 및 저장 완료!")


비둘기파 데이터에서 '한국' 포함 단어 삭제 개수: 7
매파 데이터에서 '한국' 포함 단어 삭제 개수: 0
한국 포함 단어 제거 및 저장 완료!


##### 필요없는 단어들 때문에 순도 높은 사전 구축이 되질 않는다
사전을 보고 필요없는 단어를 계속 제거

### TF-IDF 계산 ( 매파 비둘기 파 잘 구분하는가?)

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# 🔹 CSV 파일 불러오기
df_dovish = pd.read_csv("final_dovish.csv")  # 비둘기파
df_hawkish = pd.read_csv("final_hawkish.csv")  # 매파

# 🔹 단어들을 하나의 문서처럼 결합
dovish_text = ' '.join(df_dovish["Dovish"])
hawkish_text = ' '.join(df_hawkish["Hawkish"])

# 🔹 문서 리스트 생성 (비둘기파, 매파)
documents = [dovish_text, hawkish_text]

# 🔹 TF-IDF 벡터라이저 적용
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(documents)

# 🔹 단어별 TF-IDF 점수 추출
words = vectorizer.get_feature_names_out()
tfidf_scores = tfidf_matrix.toarray()

# 🔹 데이터프레임 변환
tfidf_df = pd.DataFrame(tfidf_scores.T, index=words, columns=["비둘기파", "매파"])

# 🔹 CSV로 저장
tfidf_df.to_csv("Fast-taxt_TF-IDF.csv")

print("파일 저장")


✅ TF-IDF 계산 완료! 결과가 tfidf_hawk_dove.csv 파일에 저장되었습니다.


In [ ]:
tfidf_df[:50]

In [26]:
df_dovish.columns

Index(['Dovish', 'Category'], dtype='object')

In [30]:
import pandas as pd

# 두 CSV 파일 읽기
df_dovish = pd.read_csv("final_dovish.csv")
df_hawkish = pd.read_csv("final_hawkish.csv")

# 각 데이터프레임의 'Dovish' 열과 'Hawkish' 열만 추출
dovish_words = df_dovish['Dovish']
hawkish_words = df_hawkish['Hawkish']

# 길이가 맞지 않다면 더 짧은 열을 맞추기 위해 NaN으로 채움
max_len = max(len(dovish_words), len(hawkish_words))
dovish_words = dovish_words.reindex(range(max_len))
hawkish_words = hawkish_words.reindex(range(max_len))

# 두 열을 하나의 데이터프레임으로 결합
combined_df = pd.DataFrame({
    'Dovish': dovish_words,
    'Hawkish': hawkish_words
})

# 결과 확인
print(combined_df)

# 결과를 새로운 CSV로 저장
combined_df.to_csv("combined_keywords.csv", index=False)


     Dovish Hawkish
0        하향      세력
1        합의      도발
2        부진      철강
3      상호이해      투기
4        평화       높
...     ...     ...
5963    NaN      수고
5964    NaN     셰브론
5965    NaN      처전
5966    NaN      딴판
5967    NaN      펜딩

[5968 rows x 2 columns]


### 같은 크기로 맞추기로 맞춰서 두 사전간 유사도 계산

In [40]:
import pandas as pd
import os

# 파일 경로 확인
print(os.path.exists('Fast-text_final.csv'))
print(os.path.exists('예진.csv'))

# 두 파일을 문자열 타입으로 읽어들임
df1 = pd.read_csv('Fast-text_final.csv', dtype=str)
df2 = pd.read_csv('예진.csv', dtype=str)

# 둘 중 작은 크기로 맞추기
min_length = min(len(df1), len(df2))

# 두 데이터프레임의 길이를 같게 자르기
df1 = df1.head(min_length)
df2 = df2.head(min_length)

# 컬럼명을 모두 소문자로 변환하여 일관성 맞추기
df1.columns = df1.columns.str.lower()  # df1의 컬럼명을 소문자로 변환
df2.columns = df2.columns.str.lower()  # df2의 컬럼명을 소문자로 변환

# 두 DataFrame에 공통으로 있는 컬럼만 비교
common_columns = set(df1.columns).intersection(set(df2.columns))
print("Common Columns:", common_columns)

# 자카드 유사도 계산
similarity_results = {}

for col in common_columns:
    # 각 컬럼의 값들을 집합으로 변환 (NaN 제거, 좌우 공백 제거)
    set1 = set(df1[col].dropna().str.strip())
    set2 = set(df2[col].dropna().str.strip())
    
    # 유사도 계산 전, 두 집합에 포함된 값 확인
    print(f"Comparing column: {col}")
    print(f"Set1 size: {len(set1)}, Set2 size: {len(set2)}")
    print(f"Set1 example: {list(set1)[:5]}")
    print(f"Set2 example: {list(set2)[:5]}")
    
    # 교집합과 합집합 계산
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    
    # 자카드 유사도: (교집합의 크기) / (합집합의 크기)
    jaccard_similarity = len(intersection) / len(union) if union else 0
    similarity_results[col] = jaccard_similarity

# 결과 출력
if not similarity_results:
    print("No similarity results found.")
else:
    for col, sim in similarity_results.items():
        print(f"Column: {col}, Jaccard Similarity: {sim:.9f}")


True
True
Common Columns: {'dovish', 'hawkish'}
Comparing column: dovish
Set1 size: 5586, Set2 size: 5586
Set1 example: ['준편차', '시장조사', '충당금적립전', '사옥', '유니버설']
Set2 example: ['호주 한국', '철강 생산능력', '환율 시장 개입', '전세 사기 사건', '전월 비 상승 폭']
Comparing column: hawkish
Set1 size: 5586, Set2 size: 5586
Set1 example: ['내정자', '케프투자증권', '누그러질', '계양', '트래커']
Set2 example: ['압력 시각', '하락 마땅 투자처', '개발 아파트', '지난해 업계', '운수 상하이 인터내셔널 포트']
Column: dovish, Jaccard Similarity: 0.004044217
Column: hawkish, Jaccard Similarity: 0.004766616
